In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
class ConstructDataset():
  def __init__(self, type_data, data_path, mnist_path):
    self.type_data = type_data
    self.data_path = data_path
    self.mnist_path = mnist_path
    f = np.load(os.path.join(data_path, mnist_path))
    self.x_tr = torch.from_numpy(f["x_train"])
    self.y_tr = torch.from_numpy(f["y_train"]).long()
    self.x_te = torch.from_numpy(f["x_test"])
    self.y_te = torch.from_numpy(f["y_test"]).long()
    f.close()
  def create_one_task(self, type_data, seed, t, n_tasks):   # return one task as np array
    np.random.seed(seed)
    random.seed(seed)

    if type_data == 'rotated_mnist':
      def rotate_dataset(d, rotation):
        result = torch.FloatTensor(d.size(0), 784)  # 60000*784
        tensor = transforms.ToTensor()

        for i in range(d.size(0)):
          img = Image.fromarray(d[i].numpy(), mode="L")
          result[i] = tensor(img.rotate(rotation)).view(784)
        return result     # 60000 vectors-784
      
      min_rot = 1.0 * t / n_tasks * (180.0 - 0.0) + 0.0
      max_rot = 1.0 * (t+1) / n_tasks * (180.0 - 0.0) + 0.0
      rot = random.random() * (max_rot - min_rot) + min_rot
      task_tr = [rot, rotate_dataset(self.x_tr, rot), y_tr]
      task_te = [rot, rotate_dataset(self.x_te, rot), y_te]

      return task_tr, task_te
    
    if type_data = 'permuted_mnist':
      def permute(d, permutation):   # d[i] is flattened
        res = torch.FloatTensor(d.shape[0], 784)
        for i in range(d.shape[0]):
          res[i] = d[i][permutation]
        return res

      permutation = np.random.permutation(784)
      task_tr = [permute(self.x_tr.view(self.x_tr.shape[0], 28*28), permutation), y_tr]
      task_te = [permute(self.x_te.view(self.x_te.shape[0], 28*28), permutation), y_te]

      return task_tr, task_te

  def create_several_tasks(self, num_tasks, file_name, type_data):
    tasks_tr = []
    tasks_te = []
    if file_name != "":   # if save constructed data to file
      path = file_name + "_" + str(num_tasks) + ".pt"
      if not os.path.exists(os.path.join(self.data_path, path)):
        for t in range(num_tasks):
          task_tr, task_te = self.create_one_task(type_data, t, num_tasks) 
          tasks_tr.append(task_tr)
          tasks_te.append(task_te)
        torch.save([tasks_tr, tasks_te], self.data_path + "/" + path)
      return self.data_path + "/" + path
    else:   # load all task to RAM
      if type_data == 'rotated_mnist':
        path = "mnist_rotations" + "_" + str(num_tasks) + ".pt"
        if not os.path.exists(os.path.join(self.data_path, path)):
          for t in range(num_tasks):
            task_tr, task_te = self.create_one_task(type_data, t, num_tasks) 
            tasks_tr.append(task_tr)
            tasks_te.append(task_te)
          torch.save([tasks_tr, tasks_te], self.data_path + "/" + path)
        else:
          tasks_tr, tasks_te = torch.load(self.data_path + "/" + path)
      if type_data= 'permuted_mnist':
        path = "mnist_permutations" + "_" + str(num_tasks) + ".pt"
        if not os.path.exists(os.path.join(self.data_path, path)):
          for t in range(num_tasks):
            task_tr, task_te = self.create_one_task(type_data, t, num_tasks) 
            tasks_tr.append(task_tr)
            tasks_te.append(task_te)
          torch.save([tasks_tr, tasks_te], self.data_path + "/" + path)
        else:
          tasks_tr, tasks_te = torch.load(self.data_path + "/" + path)
      return tasks_tr, tasks_te